# Configuring Ubuntu 14.04.5 or 16.04 to run LXC

## Installing LXC

These instructions have been tested on Ubuntu 16.04 LTS and Ubuntu 14.04.5 LTS. They are likely not to work with other distributions of Linux. If you do not have access to a Ubuntu Linux system, you may also follow along using a Ubuntu virtual machine running in VirtualBox or an equivalent virtual machine framework. Each LXC container will be running Ubuntu 14.04.5 LTS.

### Ubuntu 14.04.5 Only: Update LXC PPA
The version of LXC available in the Ubuntu repositories is fairly old. Get an newer version by using the LXC team's PPA.

Run

```console
user@host:~$ sudo add-apt-repository ppa:ubuntu-lxc/stable
```

and press "Enter" to continue when prompted.

### Ubuntu 14.04.5 and Ubuntu 16.04: Update Repositories and Install

Next refresh your repository information and install LXC and Git by running

```console
user@host:~$ sudo apt-get update
user@host:~$ sudo apt-get install lxc git
```

### Add LXC Networking Support

Update /etc/lxc/lxc-usernet by running: 

```console
user@host:~$ sudo nano /etc/lxc/lxc-usernet
```

Update the file to match the following, replacing user-name with your current user name.

```console
# USERNAME TYPE BRIDGE COUNT
user-name veth lxcbr0 10
```

When finished editing, use Ctrl+o to Output (save) the file, and Ctrl+x to Exit from nano. 

### Set up default LXC configuration
Create a configuration directory for LXC and copy a default configuration file into it by running

```console
user@host:~$ mkdir -p ~/.config/lxc
user@host:~$ cp /etc/lxc/default.conf ~/.config/lxc/default.conf
```

Next you'll need to set up user and group ID mappings. The mappings aren't always constant, so to make sure you're using the correct mappings, look them up using the following command, remembering to replace user-name with your current user name:

```console
user@host:~$ sudo grep user-name /etc/sub{gid,uid}
```

This is an example output of that command:

```console
/etc/subgid:user-name:165536:65536
/etc/subuid:user-name:165536:65536
```

Update the default LXC configuration file you just copied by running:

```console
user@host:~$ nano ~/.config/lxc/default.conf
```

If the output of the previous command had 100000 after your username, add the following to the end of the file:

```console
lxc.id_map = u 0 100000 65536
lxc.id_map = g 0 100000 65536
```

If the output of the previous command had 165536 after your username, add the following to the end of the file:

```console
lxc.id_map = u 0 165536 65536
lxc.id_map = g 0 165536 65536
```

### Update .local directory to be able to use defaul LXC prefix
LXC needs other users and groups, specifically the other users and groups executing inside containers, to be able to execute their own files. Make this possible by adding the execute flag to your .local directory:

```console
user@host:~$ mkdir ~/.local/
user@host:~$ chmod -R o+x ~/.local/
```

### Logout so changes can take effect
You need to completely log out of the system, not just the current terminal. 

### Log back in to test
Test that you can create, run, and attach to unprivileged LXC containers by logging back in and running:

```console
user@host:~$ lxc-create -t download -n my-container -- -d ubuntu -r trusty -a amd64
user@host:~$ lxc-start -n my-container
user@host:~$ lxc-attach -n my-container
```

If everything is configured correctly, you should be logged in as the root user of the container, at a prompt that looks like this:

```console
root@my-container:/# 
```

Type exit and hit enter to get back out of the container. Shut it down and delete the container file by running:

```console 
user@host:~$ lxc-stop -n my-container
user@host:~$ lxc-destroy -n my-container
```

You are now ready to run the hurdle build scripts.